In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPRegressor
import warnings
warnings.filterwarnings("ignore")

In [18]:
pasture_species = ['phalaris','lucerne','clover_rye','improved_pasture', 'natural_grass']

In [19]:
# display
pasture_type = 'clover_rye_grass'

In [20]:
read_file = pasture_type+'_data.csv'
df = pd.read_csv(read_file)

In [21]:
df

,3DAVG_RAIN,3DAVG_MAX_TEMP,3DAVG_MIN_TEMP,3DAVG_RH_TMAX,3DAVG_RH_TMIN,3DAVG_EVAP,3DAVG_RADIATION,3D_MIN_TEMP,3D_MAX_TEMP,5DAVG_RAIN,...,365D_MIN_TEMP,365D_MAX_TEMP,FOO,RAIN,MAX_TEMP,MIN_TEMP,RH_TMAX,RH_TMIN,EVAP,RADIATION
0,0.000000,27.500000,12.150000,40.750000,98.050000,7.850000,26.250000,11.5,32.0,0.000,...,11.5,32.0,396.0,0.0,23.0,11.5,51.6,100.0,7.8,21.2
1,0.100000,22.166667,7.400000,40.500000,97.900000,6.500000,25.733333,2.6,24.3,0.075,...,2.6,32.0,393.0,0.0,24.3,2.6,24.0,99.2,5.4,29.6
2,0.000000,34.000000,10.066667,19.700000,85.066667,7.233333,33.333333,8.2,35.1,0.060,...,2.6,35.1,379.0,0.0,35.1,11.2,20.3,86.5,7.4,32.8
3,0.000000,29.900000,12.233333,29.366667,84.400000,7.866667,28.833333,11.2,35.1,0.000,...,2.6,35.1,379.0,0.0,25.2,11.5,35.6,84.1,8.2,21.3
4,0.000000,31.466667,11.333333,24.533333,83.966667,7.100000,30.533333,9.3,33.5,0.000,...,2.6,35.1,377.0,0.0,32.2,12.4,27.5,91.7,7.1,29.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16740,0.000000,24.633333,9.433333,37.800000,93.833333,5.633333,23.333333,5.9,26.5,0.000,...,0.5,39.1,2838.0,0.0,21.3,13.0,48.2,81.5,6.3,20.1
16741,1.366667,22.900000,9.766667,48.333333,93.833333,5.633333,23.433333,4.5,27.1,0.820,...,0.5,39.1,2841.0,4.1,27.1,4.5,29.6,100.0,5.0,30.5
16742,0.000000,32.600000,12.766667,27.633333,92.100000,7.133333,31.500000,12.4,33.5,0.000,...,0.5,39.1,2793.0,0.0,33.1,12.8,26.9,92.1,7.8,32.1
16743,0.000000,36.600000,16.333333,22.766667,76.100000,12.066667,28.833333,14.7,38.5,0.000,...,0.5,39.1,2774.0,0.0,35.0,18.0,23.1,63.0,13.2,26.3


In [8]:
df = df[['RAIN', 'MAX_TEMP', 'MIN_TEMP', 'RH_TMAX',
       'RH_TMIN', 'EVAP', 'RADIATION', 'FOO']]

In [9]:
y = df['FOO']
X = df.drop(['FOO'], axis=1)

In [10]:
X = pd.get_dummies(X)

In [11]:

from sklearn.preprocessing import MinMaxScaler, StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize the target variable
scaler = StandardScaler()
#target_scaled = scaler.fit_transform(target.reshape(-1, 1))
#target_scaled = np.log(target.reshape(-1, 1))
train_target = scaler.fit_transform(y_train.values.reshape(-1, 1))
test_target = scaler.transform(y_test.values.reshape(-1, 1))

In [12]:
def eval(scaler_targ,model_in, X_train,y_train,X_test,y_test):
    
    y_train = scaler_targ.inverse_transform(y_train)
    y_test = scaler_targ.inverse_transform(y_test)
     
    # Predict the labels of the train and test sets: preds
    preds_train = scaler_targ.inverse_transform(model_in.predict(X_train).reshape(-1, 1))
    preds_test = scaler_targ.inverse_transform(model_in.predict(X_test).reshape(-1, 1))
    
    # compute the rmse: rmse
    rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))
    rmse_test = np.sqrt(mean_squared_error(y_test, preds_test))
    
    # compute the mae: mae
    mae_train = mean_absolute_error(y_train,preds_train)
    mae_test = mean_absolute_error(y_test, preds_test)
    
     # compute the r2 score
    r2_train = r2_score(y_train,preds_train)
    r2_test = r2_score(y_test,preds_test)
    
    print('*********Training*********')
    print("\tRMSE: %0.2f" % (rmse_train))
    print("\tMAE: %0.2f" % (mae_train))
    print("\tR2 score: %0.2f" % (r2_train))


    print('*********Testing*********')
    print("\tRMSE: %0.2f" % (rmse_test))
    print("\tMAE: %0.2f" % (mae_test))
    print("\tR2 score: %0.2f" % (r2_test))

    return preds_train,preds_test,rmse_train,rmse_test,mae_train,mae_test,r2_train,r2_test

### Climate only

In [14]:
#days_in_out = ['3D','5D', '7D', '10D', '14D', '30D', '90D', '180D', '365D']
XGB_rmse_all_train = ['XGBoost']
bootstra_rmse_all_train = ['Bootstrap']
RF_rmse_all_train = ['Random Forest']
LR_rmse_all_train = ['Linear Regression']
DNN_rmse_all_train = ['DNN']
stack_xgb_dnn_rmse_all_train = ['Stacking XGB DNN']
stack_xgb_rf_rmse_all_train = ['Stacking XGB RF']

XGB_mae_all_train = ['XGBoost']
bootstra_mae_all_train = ['Bootstrap']
RF_mae_all_train = ['Random Forest']
LR_mae_all_train = ['Linear Regression']
DNN_mae_all_train = ['DNN']
stack_xgb_dnn_mae_all_train = ['Stacking XGB DNN']
stack_xgb_rf_mae_all_train = ['Stacking XGB RF']

XGB_r2_all_train = ['XGBoost']
bootstra_r2_all_train = ['Bootstrap']
RF_r2_all_train = ['Random Forest']
LR_r2_all_train = ['Linear Regression']
DNN_r2_all_train = ['DNN']
stack_xgb_dnn_r2_all_train = ['Stacking XGB DNN']
stack_xgb_rf_r2_all_train = ['Stacking XGB RF']


XGB_rmse_all_test = ['XGBoost']
bootstra_rmse_all_test = ['Bootstrap']
RF_rmse_all_test = ['Random Forest']
LR_rmse_all_test = ['Linear Regression']
DNN_rmse_all_test = ['DNN']
stack_xgb_dnn_rmse_all_test = ['Stacking XGB DNN']
stack_xgb_rf_rmse_all_test = ['Stacking XGB RF']

XGB_mae_all_test = ['XGBoost']
bootstra_mae_all_test = ['Bootstrap']
RF_mae_all_test = ['Random Forest']
LR_mae_all_test = ['Linear Regression']
DNN_mae_all_test = ['DNN']
stack_xgb_dnn_mae_all_test = ['Stacking XGB DNN']
stack_xgb_rf_mae_all_test = ['Stacking XGB RF']

XGB_r2_all_test = ['XGBoost']
bootstra_r2_all_test = ['Bootstrap']
RF_r2_all_test = ['Random Forest']
LR_r2_all_test = ['Linear Regression']
DNN_r2_all_test = ['DNN']
stack_xgb_dnn_r2_all_test = ['Stacking XGB DNN']
stack_xgb_rf_r2_all_test = ['Stacking XGB RF']



X_train_climate = X_train.copy()
X_test_climate = X_test.copy()

scaler2 = StandardScaler()
X_train_climate = scaler2.fit_transform(X_train_climate)
X_test_climate = scaler2.transform(X_test_climate)

print('XGBoost')
# Instantiatethe XGBRegressor: xg_reg
xg_climate = xgb.XGBRegressor(objective='reg:squarederror', seed=42, n_estimators=30, max_depth=5, reg_lambda = 1).fit(X_train_climate, train_target)
XG_C_preds_train,XG_C_preds_test,XG_C_rmse_train,XG_C_rmse_test,XG_C_mae_train,XG_C_mae_test,XG_C_r2_train,XG_C_r2_test = eval(scaler, xg_climate, X_train_climate,train_target,X_test_climate,test_target)
XGB_rmse_all_train.append(XG_C_rmse_train)
XGB_rmse_all_test.append(XG_C_rmse_test)

XGB_mae_all_train.append(XG_C_mae_train)
XGB_mae_all_test.append(XG_C_mae_test)

XGB_r2_all_train.append(XG_C_r2_train)
XGB_r2_all_test.append(XG_C_r2_test)

from sklearn.ensemble import BaggingRegressor
print('Bootstrapped Aggregating (Bagging)')
base_esti = xgb.XGBRegressor(objective='reg:squarederror', seed=42, n_estimators=30, max_depth=5, reg_lambda = 1)
n_estis = 50
bagging_model = BaggingRegressor(estimator=base_esti, n_estimators=n_estis, random_state=42)
bagging_model.fit(X_train_climate, train_target)
BXG_C_preds_train,BXG_C_preds_test,BXG_C_rmse_train,BXG_C_rmse_test,BXG_C_mae_train,BXG_C_mae_test,BXG_C_r2_train,BXG_C_r2_test = eval(scaler, bagging_model,  X_train_climate,train_target,X_test_climate,test_target)
bootstra_rmse_all_train.append(BXG_C_rmse_train)
bootstra_rmse_all_test.append(BXG_C_rmse_test)
bootstra_mae_all_train.append(BXG_C_mae_train)
bootstra_mae_all_test.append(BXG_C_mae_test)
bootstra_r2_all_train.append(BXG_C_r2_train)
bootstra_r2_all_test.append(BXG_C_r2_test)

from sklearn.ensemble import RandomForestRegressor
print('Random Forest')
# Instantiatethe XGBRegressor: xg_reg
rf_climate = RandomForestRegressor(max_depth=5, random_state = 5).fit(X_train_climate, train_target)
rf_C_preds_train,rf_C_preds_test,rf_C_rmse_train,rf_C_rmse_test,rf_C_mae_train,rf_C_mae_test,rf_C_r2_train,rf_C_r2_test = eval(scaler, rf_climate,  X_train_climate,train_target,X_test_climate,test_target)
RF_rmse_all_train.append(rf_C_rmse_train)
RF_rmse_all_test.append(rf_C_rmse_test)
RF_mae_all_train.append(rf_C_mae_train)
RF_mae_all_test.append(rf_C_mae_test)
RF_r2_all_train.append(rf_C_r2_train)
RF_r2_all_test.append(rf_C_r2_test)


from sklearn.linear_model import LinearRegression
print('Linear Regression')
lr_climate = LinearRegression().fit(X_train_climate, train_target)
lr_C_preds_train,lr_C_preds_test,lr_C_rmse_train,lr_C_rmse_test,lr_C_mae_train,lr_C_mae_test,lr_C_r2_train,lr_C_r2_test = eval(scaler, lr_climate,  X_train_climate,train_target,X_test_climate,test_target)
LR_rmse_all_train.append(lr_C_rmse_train)
LR_rmse_all_test.append(lr_C_rmse_test)
LR_mae_all_train.append(lr_C_mae_train)
LR_mae_all_test.append(lr_C_mae_test)
LR_r2_all_train.append(lr_C_r2_train)
LR_r2_all_test.append(lr_C_r2_test)


print('DNN')
dnn_climate = MLPRegressor(random_state=1, max_iter=1000, hidden_layer_sizes=(75,75)).fit(X_train_climate, np.array(train_target).reshape(-1, ))
DNN_C_preds_train,DNN_C_preds_test,DNN_C_rmse_train,DNN_C_rmse_test,DNN_C_mae_train,DNN_C_mae_test,DNN_C_r2_train,DNN_C_r2_test = eval(scaler, dnn_climate, X_train_climate,train_target,X_test_climate,test_target)
DNN_rmse_all_train.append(DNN_C_rmse_train)
DNN_rmse_all_test.append(DNN_C_rmse_test)
DNN_mae_all_train.append(DNN_C_mae_train)
DNN_mae_all_test.append(DNN_C_mae_test)
DNN_r2_all_train.append(DNN_C_r2_train)
DNN_r2_all_test.append(DNN_C_r2_test)


print('Stacking XGBoost + DNN')
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
base_models = [
    ('XGBoost', xgb.XGBRegressor(objective='reg:squarederror', seed=42, n_estimators=30, max_depth=5, reg_lambda = 1)),
    ('DNN', MLPRegressor(random_state=1, max_iter=1000, hidden_layer_sizes=(75,75)))
]
# Define the meta-model (final estimator)
meta_model = LinearRegression()
# Create the stacking model
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_model)
# Train the stacking model
stacking_regressor.fit(X_train_climate, train_target)
stack_C_preds_train,stack_C_preds_test,stack_C_rmse_train,stack_C_rmse_test,stack_C_mae_train,stack_C_mae_test,stack_C_r2_train,stack_C_r2_test = eval(scaler,stacking_regressor, X_train_climate,train_target,X_test_climate,test_target)

stack_xgb_dnn_rmse_all_train.append(stack_C_rmse_train)
stack_xgb_dnn_rmse_all_test.append(stack_C_rmse_test)
stack_xgb_dnn_mae_all_train.append(stack_C_mae_train)
stack_xgb_dnn_mae_all_test.append(stack_C_mae_test)
stack_xgb_dnn_r2_all_train.append(stack_C_r2_train)
stack_xgb_dnn_r2_all_test.append(stack_C_r2_test)

print('Stacking XGBoost + RF')
base_models_xr = [
    ('XGBoost', xgb.XGBRegressor(objective='reg:squarederror', seed=42, n_estimators=30, max_depth=5, reg_lambda = 1)),
    ('RF', RandomForestRegressor(max_depth=5, random_state = 5))
]
# Define the meta-model (final estimator)
meta_model_xr = MLPRegressor(random_state=1, max_iter=1000, hidden_layer_sizes=(75,75))
# Create the stacking model
stacking_regressor_xr = StackingRegressor(estimators=base_models_xr, final_estimator=meta_model_xr)
# Train the stacking model
stacking_regressor_xr.fit(X_train_climate, train_target)
stack_xr_C_preds_train,stack_xr_C_preds_test,stack_xr_C_rmse_train,stack_xr_C_rmse_test,stack_xr_C_mae_train,stack_xr_C_mae_test,stack_xr_C_r2_train,stack_xr_C_r2_test = eval(scaler,stacking_regressor_xr, X_train_climate,train_target,X_test_climate,test_target)
stack_xgb_rf_rmse_all_train.append(stack_xr_C_rmse_train)
stack_xgb_rf_rmse_all_test.append(stack_xr_C_rmse_test)
stack_xgb_rf_mae_all_train.append(stack_xr_C_mae_train)
stack_xgb_rf_mae_all_test.append(stack_xr_C_mae_test)
stack_xgb_rf_r2_all_train.append(stack_xr_C_r2_train)
stack_xgb_rf_r2_all_test.append(stack_xr_C_r2_test)


XGBoost
*********Training*********
	RMSE: 303.97
	MAE: 189.11
	R2 score: 0.92
*********Testing*********
	RMSE: 304.96
	MAE: 193.20
	R2 score: 0.91
Bootstrapped Aggregating (Bagging)
*********Training*********
	RMSE: 304.19
	MAE: 188.66
	R2 score: 0.92
*********Testing*********
	RMSE: 303.94
	MAE: 191.87
	R2 score: 0.92
Random Forest
*********Training*********
	RMSE: 546.56
	MAE: 367.88
	R2 score: 0.73
*********Testing*********
	RMSE: 531.44
	MAE: 367.15
	R2 score: 0.74
Linear Regression
*********Training*********
	RMSE: 880.35
	MAE: 660.48
	R2 score: 0.29
*********Testing*********
	RMSE: 872.77
	MAE: 665.12
	R2 score: 0.30
DNN
*********Training*********
	RMSE: 315.86
	MAE: 197.38
	R2 score: 0.91
*********Testing*********
	RMSE: 317.71
	MAE: 202.21
	R2 score: 0.91
Stacking XGBoost + DNN
*********Training*********
	RMSE: 298.24
	MAE: 182.81
	R2 score: 0.92
*********Testing*********
	RMSE: 300.04
	MAE: 187.04
	R2 score: 0.92
Stacking XGBoost + RF
*********Training*********
	RMSE: 297.71
	

In [15]:
df_RMSE_training = [XGB_rmse_all_train,bootstra_rmse_all_train,RF_rmse_all_train,LR_rmse_all_train,DNN_rmse_all_train,stack_xgb_dnn_rmse_all_train,stack_xgb_rf_rmse_all_train]

In [16]:
df_RMSE_training

[['XGBoost', 303.9670438179384],
 ['Bootstrap', 304.1896260705098],
 ['Random Forest', 546.5564657283804],
 ['Linear Regression', 880.3513570339277],
 ['DNN', 315.86238015374806],
 ['Stacking XGB DNN', 298.2444222492617],
 ['Stacking XGB RF', 297.71275650636613]]

In [17]:

import pandas as pd
 
# List
df_RMSE_training = [XGB_rmse_all_train,bootstra_rmse_all_train,RF_rmse_all_train,LR_rmse_all_train,DNN_rmse_all_train,stack_xgb_dnn_rmse_all_train,stack_xgb_rf_rmse_all_train]
df_MAE_training = [XGB_mae_all_train,bootstra_mae_all_train,RF_mae_all_train,LR_mae_all_train,DNN_mae_all_train,stack_xgb_dnn_mae_all_train,stack_xgb_rf_mae_all_train]
df_R2_training = [XGB_r2_all_train,bootstra_r2_all_train,RF_r2_all_train,LR_r2_all_train,DNN_r2_all_train,stack_xgb_dnn_r2_all_train,stack_xgb_rf_r2_all_train]


df_RMSE_test = [XGB_rmse_all_test,bootstra_rmse_all_test,RF_rmse_all_test,LR_rmse_all_test,DNN_rmse_all_test,stack_xgb_dnn_rmse_all_test,stack_xgb_rf_rmse_all_test]
df_MAE_test = [XGB_mae_all_test,bootstra_mae_all_test,RF_mae_all_test,LR_mae_all_test,DNN_mae_all_test,stack_xgb_dnn_mae_all_test,stack_xgb_rf_mae_all_test]
df_R2_test = [XGB_r2_all_test,bootstra_r2_all_test,RF_r2_all_test,LR_r2_all_test,DNN_r2_all_test,stack_xgb_dnn_r2_all_test,stack_xgb_rf_r2_all_test]
 
#Create a DataFrame object
df_RMSE_training_output = pd.DataFrame(df_RMSE_training, 
                  columns = ['Model','0D']) 
df_RMSE_test_output = pd.DataFrame(df_RMSE_test, 
                  columns = ['Model','0D']) 
              
df_MAE_training_output = pd.DataFrame(df_MAE_training, 
                  columns = ['Model','0D']) 
df_MAE_test_output = pd.DataFrame(df_MAE_test, 
                  columns = ['Model','0D']) 
              
df_R2_training_output = pd.DataFrame(df_R2_training, 
                  columns = ['Model','0D']) 
df_R2_test_output = pd.DataFrame(df_R2_test, 
                  columns = ['Model','0D']) 
 
# display
pasture_type = pasture_type+'_0D'
print('RMSE')
display(df_RMSE_training_output)
df_RMSE_training_output.to_csv('rmse_training_'+pasture_type+'.csv', index = False)
display(df_RMSE_test_output)
df_RMSE_test_output.to_csv('rmse_testing_'+pasture_type+'.csv', index = False)
print('MAE')
display(df_MAE_training_output)
df_MAE_training_output.to_csv('mae_training_'+pasture_type+'.csv', index = False)
display(df_MAE_test_output)
df_MAE_test_output.to_csv('mae_test_'+pasture_type+'.csv', index = False)
print('R2')
display(df_R2_training_output)
df_MAE_training_output.to_csv('r2_training_'+pasture_type+'.csv', index = False)
display(df_R2_test_output)
df_MAE_test_output.to_csv('r2_test_'+pasture_type+'.csv', index = False)

RMSE


,Model,0D
0,XGBoost,303.967044
1,Bootstrap,304.189626
2,Random Forest,546.556466
3,Linear Regression,880.351357
4,DNN,315.862380
5,Stacking XGB DNN,298.244422
6,Stacking XGB RF,297.712757


,Model,0D
0,XGBoost,304.956219
1,Bootstrap,303.943235
2,Random Forest,531.435146
3,Linear Regression,872.768610
4,DNN,317.714200
5,Stacking XGB DNN,300.035824
6,Stacking XGB RF,300.071120


MAE


,Model,0D
0,XGBoost,189.111918
1,Bootstrap,188.663041
2,Random Forest,367.876309
3,Linear Regression,660.478273
4,DNN,197.383751
5,Stacking XGB DNN,182.806636
6,Stacking XGB RF,182.768024


,Model,0D
0,XGBoost,193.196006
1,Bootstrap,191.870590
2,Random Forest,367.152763
3,Linear Regression,665.123490
4,DNN,202.207211
5,Stacking XGB DNN,187.040368
6,Stacking XGB RF,187.104999


R2


,Model,0D
0,XGBoost,0.915590
1,Bootstrap,0.915466
2,Random Forest,0.727095
3,Linear Regression,0.291968
4,DNN,0.908854
5,Stacking XGB DNN,0.918738
6,Stacking XGB RF,0.919028


,Model,0D
0,XGBoost,0.914687
1,Bootstrap,0.915253
2,Random Forest,0.740917
3,Linear Regression,0.301227
4,DNN,0.907400
5,Stacking XGB DNN,0.917418
6,Stacking XGB RF,0.917399


----------------------------------------------------------------------------------------------------------------------